<a href="https://colab.research.google.com/github/TheAgaveFairy/MusicGeneration/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/arnavmishra6996/music-generation-using-decoder-only-model-gpt2

this is the inspiration, more or less copying it but in my own typing + modifications so i understand it

In [6]:
import numpy as np
import pandas as pd
import os
import transformers

from helpers import list_files

In [22]:
def augment_data(midi_df):
    TRANSPOSE_RANGE = 3
    TEMPO_RANGE = 0.15
    VELOCITY_RANGE = 5

    transpose = np.random.randint(-TRANSPOSE_RANGE, TRANSPOSE_RANGE)
    tempo = np.random.uniform(1 - TEMPO_RANGE, 1 + TEMPO_RANGE)
    velocity = np.random.randint(-VELOCITY_RANGE, VELOCITY_RANGE)
    #print(f"trans: {transpose}, tempo: {tempo}")

    if np.random.random() > 0.25:
        print(f"HRT time: transing {transpose} steps")
        df['note_value'] = df['note_value'] + transpose
        
    if np.random.random() > 0.25:
        print(f"codeine time: tempo change {tempo}")
        df['start_time'] = df['start_time'] * tempo
        df['end_time'] = df['end_time'] * tempo

    if np.random.random() > 0.25:
        print(f"bad musician: velocity {velocity}")
        df['velocity'] = df['velocity'] + velocity

    return df # but isnt this already doing it in place

In [ ]:
def data_generator():
    
    yield midi_as_text

In [26]:
test_folder_path = './inputs/final/Debussy'
test_file_name = os.listdir(test_folder_path)[0]
test_file_path = os.path.join(test_folder_path, test_file_name)

dataset = tf.data.Dataset.from_tensor_slices(os.listdir(test_folder_path))
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
df = pd.read_csv(test_file_path)
print(df.head())

augment_data(df)
print(df.head())

GPT-2 needs this as text

In [2]:
def tokenize_midi(midi_df):
    """
    with open('midi_text_data.txt', 'w') as file:
        file.write('<|startoftext|>\n')
        for index, row in tqdm(df.iterrows()):
            text_line = f"start_time: {row['start_time']}, end_time: {row['end_time']}, instrument: {row['instrument']}, note: {row['note']}, start_beat: {row['start_beat']}, end_beat: {row['end_beat']}, note_value: {row['note_value']}, duration: {row['duration']}, velocity: {row['velocity']}\n"
            file.write(text_line)
        file.write('<|endoftext|>\n')
    """
    return None

Here we go

In [3]:
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            input_ids = ids[i:i+block_size]
            labels = ids[i+1:i+block_size+1] # shifted by 1
            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100

            yield {
                "input_ids": input_ids,#ids[i:i+block_size],
                "attention_mask": [1] * block_size,
                "labels": labels
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(block_size,), dtype=tf.int32)
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset('midi_text_data.txt', tokenizer)

2025-04-03 14:58:57.025290: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 14:58:57.116639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743710337.153922   75106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743710337.164525   75106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 14:58:57.250726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
tf.config.list_physical_devices('CPU'),tf.config.list_physical_devices('GPU')

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')],
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

In [ ]:
"""
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, TFTrainingArguments #TFTrainer,
from transformers import DataCollatorForLanguageModeling
from transformers import TFTrainer# as TFTrainer
from tqdm import tqdm

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define the training arguments
training_args = TFTrainingArguments(
    output_dir="./gpt2-midi",  # Directory where the model and tokenizer will be saved
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    save_steps=10_000,  # Save every 10,000 steps
    save_total_limit=2,  # Keep only the 2 most recent models
    prediction_loss_only=True,  # Only compute the loss during training
    logging_dir='./logs',  # Directory for logs
    logging_steps=500  # Log every 500 steps
)

#Gemini
strategy = tf.distribute.MirroredStrategy(['CPU:0'])

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    #/Gemini

# Initialize the Trainer
trainer = TFTrainer(
    model=model,#TFGPT2LMHeadModel.from_pretrained('gpt2'),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)
"""
'This code was deprecated on or about 2021.'

In [ ]:
"""
# Start training
trainer.train()

# Save the model and tokenizer
trainer.save_model('./gpt2-midi')
tokenizer.save_pretrained('./gpt2-midi')
"""

In [8]:
#Claude 3.5Sonnet to the rescue, maybe
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling, DefaultDataCollator
from tqdm import tqdm
import os

# Initialize tokenizer and data collator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
"""
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True # changed from False
)
"""
data_collator = DefaultDataCollator()

# Training configuration
class TrainingConfig:
    output_dir = "./gpt2-midi"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    save_steps = 10_000
    save_total_limit = 2
    logging_dir = './logs'
    logging_steps = 500

config = TrainingConfig()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

# Set up distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')

    # Set up the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss
    )

# Prepare the dataset
def prepare_dataset_for_training(dataset, batch_size):
    """Convert dataset to tf.data.Dataset format"""
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    """
    # Modify attention mask shape
    dataset = dataset.map(lambda x: {
        **x,
        "attention_mask": tf.reshape(x["attention_mask"], (tf.shape(x["attention_mask"])[0], 1, 1, tf.shape(x["attention_mask"])[2]))  # Reshape to (batch_size, 1, 1, seq_len)
    })
    """
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset_for_training(
    train_dataset,  # Your original train_dataset
    config.per_device_train_batch_size
)

# Custom callback for model saving
class ModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_steps, output_dir, save_total_limit):
        super().__init__()
        self.save_steps = save_steps
        self.output_dir = output_dir
        self.save_total_limit = save_total_limit
        self.step = 0

    def on_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step % self.save_steps == 0:
            # Save the model
            checkpoint_dir = os.path.join(
                self.output_dir,
                f"checkpoint-{self.step}"
            )
            self.model.save_pretrained(checkpoint_dir)

            # Remove old checkpoints if exceeding limit
            checkpoints = sorted([
                d for d in os.listdir(self.output_dir)
                if d.startswith("checkpoint-")
            ])
            while len(checkpoints) > self.save_total_limit:
                checkpoint_to_remove = os.path.join(
                    self.output_dir,
                    checkpoints.pop(0)
                )
                tf.io.gfile.rmtree(checkpoint_to_remove)

# Set up callbacks
callbacks = [
    ModelCheckpoint(
        save_steps=config.save_steps,
        output_dir=config.output_dir,
        save_total_limit=config.save_total_limit
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=config.logging_dir,
        update_freq=config.logging_steps
    )
]

# Training
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=config.num_train_epochs,
    callbacks=callbacks
)

# Save the final model and tokenizer
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)
print(f"Model saved to {config.output_dir}")

2025-02-14 10:02:30.299578: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 10:02:30.391479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739548950.431066   11259 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739548950.442127   11259 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 10:02:30.530080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


I0000 00:00:1739548952.842162   11259 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5616 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


NameError: name 'train_dataset' is not defined